# Aprenentage Supervisat - Classificació
**Descripció**. 
Anem a practicar i a familiaritzar-nos amb algoritmes de classificació.

## Nivell 1
### Exercici 1
Crea almenys tres models de classificació diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.

### Exercici 2
Crea una nova variable segons si el vol ha arribat tard o no (ArrDelay > 0).

### Exercici 3
Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.

### Exercici 4
Entrena’ls utilitzant els diferents paràmetres que admeten.

### Exercici 5
Compara el seu rendiment utilitzant l’aproximació traint/test o utilitzant totes les dades (validació interna).

## Nivell 2
### Exercici 6
Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

## Nivell 3
### Exercici 7
No utilitzis la variable DepDelay a l’hora de fer prediccions